In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

import sys
import os
sys.path.append(os.path.abspath(os.path.join('../scripts')))

from featSelectFunctions.LassoVarSelectCV import LassoVarSelectCV
from featSelectFunctions.EnetVarSelectCV import EnetVarSelectCV
from KfoldLinearModels import KfoldLinearRegression

import os


In [ ]:
def rrmse(obs, Pred):
    return np.sqrt(np.mean(((obs - Pred)/obs)**2))

def rrmse(obs, Pred):
    return np.sqrt(np.mean(((obs - Pred)/obs)**2))

def rmse(obs, Pred):
    return np.sqrt(np.mean(((obs - Pred))**2))

factkm = (1000)/(60*60)



In [ ]:
Root_data_path = os.path.abspath(os.path.join('../Data'))
data = pd.read_csv(os.path.join(Root_data_path ,"data_train.csv"))

data = pd.read_csv(os.path.join(Root_data_path, "data_train.csv"))

Response = ['1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14']



Explanatory = np.array(['EPSG3979_x', 'EPSG3979_y', 'ASP100m.tif', 'ASP500m.tif',
       'ASP1000m.tif', 'ASP1500m.tif', 'ASP2000m.tif', 
       'MXC100m.tif', 'MXC500m.tif', 'MXC1000m.tif', 'MXC1500m.tif',
       'MXC2000m.tif', 'MNC100m.tif', 'MNC500m.tif', 'MNC1000m.tif',
       'MNC1500m.tif', 'MNC2000m.tif', 'RGI100m.tif', 'RGI500m.tif',
       'RGI1000m.tif', 'RGI1500m.tif', 'RGI2000m.tif', 'SLP100m.tif',
       'SLP500m.tif', 'SLP1000m.tif', 'SLP1500m.tif', 'SLP2000m.tif',
       'DVE100m.tif', 'DVE500m.tif', 'DVE1000m.tif',
       'DVE1500m.tif', 'DVE2000m.tif', 'DFE100m.tif', 'DFE500m.tif',
       'DFE1000m.tif', 'DFE1500m.tif', 'DFE2000m.tif', 'EPC100m.tif',
       'EPC500m.tif', 'EPC1000m.tif', 'EPC1500m.tif', 'EPC2000m.tif',
       'RTP100m.tif', 'RTP500m.tif', 'RTP1000m.tif', 'RTP1500m.tif',
       'RTP2000m.tif','DoG_1_5.tif', 'DoG_1_10.tif', 'DoG_5_10.tif', 'DoG_3_5.tif',
       'DoG_10_20.tif', 'DoG_10_15.tif', 'DoG_1_20.tif', 'DoG_5_20.tif',
       'LoG_1m.tif', 'LoG_5m.tif', 'LoG_10m.tif', 'LoG_15m.tif',
       'LoG_20m.tif', 'GCV100m.tif', 'GCV500m.tif', 'GCV1000m.tif',
       'GCV1500m.tif', 'GCV2000m.tif', 'ALOS_100m.tif',
       'MINC100m.tif', 'MINC500m.tif', 'MINC1000m.tif', 'MINC1500m.tif',
       'MINC2000m.tif', 'PCUR1000m.tif', 'PCUR1500m.tif', 'PCUR2000m.tif',
       'PCUR100m.tif', 'PCUR500m.tif', 'TCUR100m.tif', 'TCUR500m.tif',
       'TCUR1000m.tif', 'TCUR1500m.tif', 'TCUR2000m.tif', 'SDS100m.tif',
       'SDS500m.tif', 'SDS1000m.tif', 'SDS1500m.tif', 'SDS2000m.tif',
       'SAR100m.tif', 'SAR500m.tif', 'SAR1000m.tif', 'SAR1500m.tif',
       'SAR2000m.tif', 'TOCUR100m.tif', 'TOCUR500m.tif', 'TOCUR1000m.tif',
       'TOCUR1500m.tif', 'TOCUR2000m.tif', 'RoughL_100m', 'RoughL_1km',
       'RoughL_1km5',  'RoughL_2km', 
       'RoughL_500m',  'SeaDist', 'GMP_1', 'GMP_2', 'GMP_3',
       'GMP_5', 'GMP_6', 'GMP_7', 'GMP_8', 'GMP_9', 'GMP_10', 
       'PLC_5', 'PLC_6', 'PLC_7'])

# Enet 

In [ ]:
def bestEnetPara(data, varY, VarX, alphas=np.linspace(0.1, 0.8, 10), L1ratios= np.linspace(0.4, 1, 10), niter=50):
    AllRes = []
    EnetPara = []
    for a in alphas:
        al = [a]
        for b in L1ratios:
            l1r = [b]
            res = []
            
            EnetPara.append((a, b))
            for _ in range(niter):
                EnetvarX, _ = EnetVarSelectCV(data, varY, VarX, al , l1r)
                if len(EnetvarX) == 0:
                    res.append(-99999)
                else:
                    res.append(KfoldLinearRegression(data, varY, EnetvarX)["r2"])
            AllRes.append(res)

    
    ResBest = [[], [], []]

    for res, a in zip(AllRes, EnetPara):
        ResBest[0].append(a); ResBest[1].append(np.mean(res)); ResBest[2].append(np.min(res))
        
    ResBest = pd.DataFrame(np.array(ResBest).T, columns=["EnetPara", "Mean", "Min"])
    return ResBest


In [ ]:
EnetPara = []

for varY in Response:

    Result = bestEnetPara(data, varY=varY, VarX=Explanatory)
    
    BestPara = Result.sort_values("Mean").iloc[-1, ]["EnetPara"]
    EnetPara.append(BestPara)

# LASSO

In [ ]:
def bestAlphaLasso(data, varY, VarX, alphas=np.linspace(0.1, 0.5, 50), niter=50):
    AllRes = []
    
    for a in alphas:
        para = [a]
        
        res = []
        for _ in range(niter):
            LvarX = LassoVarSelectCV(data, varY, VarX, para)
            if len(LvarX) == 0:
                res.append(99999)
            else:
                res.append(KfoldLinearRegression(data, varY, LvarX)["rrmse"])
        AllRes.append(res)

    
    ResBest = [[], [], []]

    for res, a in zip(AllRes, alphas):
        ResBest[0].append(a); ResBest[1].append(np.mean(res)); ResBest[2].append(np.max(res))
        
    ResBest = pd.DataFrame(np.array(ResBest).T, columns=["alpha", "Mean", "Max"])
    return ResBest


In [ ]:
LassoAlpha = []

for varY in Response:

    ResLasso = bestAlphaLasso(data, varY, Explanatory)
    LassoAlpha.append(ResLasso.sort_values(by="Max").iloc[0, 0])